In [2]:
import re
import numpy as np
import pandas as pd
import json
import shutil
import gzip
import os
import wget
import tarfile
import time
from time_converter import Time

- We will start with the users,receipts and the brands last
- We will look at the dataset and create structured data model

# User Data Cleaning

In [3]:
user_df = pd.read_csv('csv_files/users.csv', index_col = 0)

In [4]:
users_df = user_df.copy()

In [5]:
users_df.columns

Index(['_id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource',
       'state'],
      dtype='object')

- We have a few Dtype objects in our dataset to observe for. While also checking the int64 and float64

In [6]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 27.6+ KB


- Checking first 5 rows we see a few mor data that is in need of cleaning

In [7]:
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [8]:
id_list = list(users_df[f'lastLogin'].to_dict().values())
date_parsed = id_list[0].replace('}', "").split(':')[1].replace(" ", "")
date_converted = Time(int(date_parsed[:10]), 'posix').to('dt')
format_date = "%m/%d/%Y"
format_time = "%H:%M:%S"
formated_date = date_converted.strftime(format_date)
formated_time = date_converted.strftime(format_time)

In [9]:
def cleanDateLog(id_data, column=None):
    id_list = list(id_data[f'{column}'].to_dict().values())
    times_list = []
    dates_list = []
    for i in range(len(id_list)):
        try: 
            
            date_parsed = id_list[i].replace('}', "").split(':')[1].replace(" ", "")
            date_converted = Time(int(date_parsed[:10]), 'posix').to('dt')
            format_date = "%m/%d/%Y"
            format_time = "%H:%M:%S"
            formated_date = date_converted.strftime(format_date)
            formated_time = date_converted.strftime(format_time)
            times_list.append(formated_time)
            dates_list.append(formated_date)
        except AttributeError:
            
            date_parsed = str(id_list[i])
            dates_list.append(date_parsed)
            
    return dates_list

In [10]:
def cleanIdCpg(id_data, column=None):
#     print(id_data[f'{column}'])
    # create the dictionary into a list to only get the values
    id_list = list(id_data[f'{column}'].to_dict().values())
    cleanedList = []
    for i in range(len(id_list)):
        parsed_name = id_list[i]
        # remove commas colons and split at quotations
        parsed_names = re.findall("'([^']*)'", id_list[i])
        # these were reoccuring strings that i saw
        cleanedList.append(parsed_names[-1])

    return cleanedList

In [11]:
users_df = user_df.copy()

users_df['_id'] = cleanIdCpg(id_data=users_df, column="_id")
users_df['createdDate'] = cleanDateLog(id_data=users_df, column="createdDate")
users_df["lastLogin"] = cleanDateLog(id_data=users_df, column="lastLogin")

# convert to pandas datetime 
users_df['createdDate'] = pd.to_datetime(users_df['createdDate'])
users_df["lastLogin"] = pd.to_datetime(users_df['lastLogin'])
users_df = users_df.rename(columns = {'_id':'user_id'}, inplace = False)

In [12]:
users_df.head()

,user_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03,2021-01-03,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI


# Receipt Data Cleaning

In [25]:
# I observed and found thats needed to converted a quick script to loop through the Dates
def convertDates(df):
    new_df = df.copy()
    col_list = list(new_df.columns)
    dates_col = col_list[3:10]
    dates_col.remove(col_list[8])
#     print(dates_col)
    for dates in dates_col:
        df[f'{dates}'] = cleanDateLog(id_data=df, column=f"{dates}")
        
    return df

# receipts_df = convertDates(receipt_df)

In [73]:
receipt_df = pd.read_csv('csv_files/receipts.csv', index_col = 0)
receipts_df = convertDates(receipt_df)
receipts_df['_id'] = cleanIdCpg(receipts_df, column = '_id')
# receipts_df = receipts_df.drop('rewardsReceiptItemList', axis = 1)

In [24]:
receipts_df

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",01/03/2021,01/03/2021,01/03/2021,01/03/2021,01/03/2021,500.0,01/03/2021,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",01/03/2021,01/03/2021,01/03/2021,01/03/2021,01/03/2021,150.0,01/02/2021,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,01/03/2021,01/03/2021,nan,01/03/2021,nan,5.0,01/03/2021,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,01/03/2021,01/03/2021,01/03/2021,01/03/2021,01/03/2021,5.0,01/03/2021,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,01/03/2021,01/03/2021,01/03/2021,01/03/2021,01/03/2021,5.0,01/02/2021,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,03/01/2021,03/01/2021,nan,03/01/2021,nan,25.0,08/17/2020,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,03/01/2021,03/01/2021,nan,03/01/2021,nan,NaN,nan,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,03/01/2021,03/01/2021,nan,03/01/2021,nan,NaN,nan,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,03/01/2021,03/01/2021,nan,03/01/2021,nan,25.0,08/17/2020,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [18]:
total = len(pd.read_json('json_files/receipts.json', lines = True)['rewardsReceiptItemList'])

- We can find that there are some nested data I tried different ways extract the information 
    felt this would be a proper way.
    - Observing the unstructured data "barcode" and "UserFlaggedBarcode" match 
    - Observing the unstructured data "description" and "Userdesription" match as well
    (We could use NLP to do further research on percentage of matching words)
- Note: I am going to flatten the data and create a fucntion to append the data to a new dataframe
        - I will also overwrite the nan values on barcode description and finalprice to condense the data
            - if we need too add any more we can in the future. 

    

In [93]:
Items_df = pd.read_json('json_files/receipts.json', lines = True)

In [111]:
total  = len(Items_df)
range(total)[:4]

range(0, 4)

In [95]:
receiptItem_df = Items_df['rewardsReceiptItemList']

In [100]:
nested_receiptItems = len(receiptItem_df[75])

- Note to self 12:33PM 
 - Fix the script to a different order with an if statement at the end if statement

In [423]:
sample_lst = []
lst_labels = []
missing_data = []
for i in range(total):
    receiptItem_df = Items_df['rewardsReceiptItemList'][i]
    # incase we have keyerrors that could mean its only one 
    try:
        total_items = len(receiptItem_df)
        for j in range(total_items):
            if j > 1:
#                 print(f'{receiptItem_df[j]}')
                lst_labels.append(i)
#                 print(i)
#                 print(j)
                sample_lst.append(receiptItem_df)
            else:
#                 print(f'something - {receiptItem_df[j]}')
                lst_labels.append(i)
#                 print(i)
#                 print(j)
                sample_lst.append(receiptItem_df)
    except:
#         print(f'Found Key error {receiptItem_df}')
        missing_data.append(receiptItem_df)
    
    else:
        print(f'next {i}')
#         sample_lst.append(receiptItem_df)
#         lst_labels.append(i)
sample_df_0 = pd.json_normalize(pd.DataFrame(sample_lst)[0])
sample_df_1 = pd.DataFrame(sample_lst[1])
sample_concat = pd.concat([sample_df_0,sample_df_1])
sample_concat.head()

next 0
next 1
next 2
next 3
next 4
next 5
next 6
next 7
next 8
next 9
next 10
next 11
next 12
next 13
next 14
next 15
next 16
next 17
next 18
next 19
next 20
next 21
next 22
next 23
next 24
next 25
next 26
next 27
next 28
next 29
next 30
next 31
next 32
next 33
next 34
next 35
next 36
next 37
next 38
next 39
next 40
next 41
next 42
next 43
next 44
next 45
next 46
next 47
next 48
next 49
next 50
next 51
next 52
next 53
next 54
next 55
next 56
next 57
next 58
next 59
next 60
next 61
next 62
next 63
next 64
next 65
next 66
next 67
next 68
next 69
next 70
next 72
next 73
next 74
next 75
next 76
next 77
next 78
next 79
next 80
next 81
next 82
next 83
next 84
next 85
next 86
next 87
next 88
next 89
next 90
next 91
next 92
next 94
next 95
next 96
next 97
next 98
next 99
next 100
next 101
next 102
next 103
next 104
next 105
next 106
next 107
next 108
next 109
next 110
next 111
next 112
next 113
next 114
next 115
next 116
next 117
next 118
next 119
next 120
next 121
next 122
next 123
next 124
n

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,needsFetchReviewReason,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,userFlaggedDescription,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [424]:
len(missing_data)
len(sample_lst)

440

In [426]:
sample_df_0 = pd.json_normalize(pd.DataFrame(sample_lst)[0])
sample_df_1 = pd.DataFrame(sample_lst[1])
sample_concat = pd.concat([sample_df_0,sample_df_1])

In [492]:
test_1 = sample_concat
test_1['barcode'] = test_1['barcode'].replace(np.nan, '4011')
test_1['description'] = test_1['description'].replace(np.nan, 'ITEM NOT FOUND')
test_1['needsFetchReview'] = test_1['needsFetchReview'].replace(np.nan, False)
test_1['finalPrice'] = test_1['finalPrice'].replace(np.nan, 0)
test_1['itemPrice'] = test_1['itemPrice'].replace(np.nan, 0)
test_1['needsFetchReview'] = test_1['needsFetchReview'].replace(np.nan, False)
test_1['needsFetchReview'] = test_1['needsFetchReview'].replace(np.nan, False)
test_1['needsFetchReview'] = test_1['needsFetchReview'].replace(np.nan, False)

In [357]:
sample_concat['barcode']

0                NaN
1                NaN
2                NaN
3                NaN
4               4011
            ...     
6938      B076FJ92M4
6939      B076FJ92M4
6940      B076FJ92M4
0                NaN
1       028400642255
Name: barcode, Length: 6943, dtype: object

In [391]:
new_data.columns

Index(['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'originalMetaBriteBarcode',
       'originalMetaBriteDescription', 'pointsNotAwardedReason',
       'pointsPayerId', 'rewardsGroup', 'rewardsProductPartnerId', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber', 'needsFetchReviewReason',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'userFlaggedDescription', 'deleted',
       'priceAfterCoupon', 'metabriteCampaignId'],
      dtype='object')

In [470]:
data['barcode'].isna().value_counts()

False    2930
Name: barcode, dtype: int64

In [465]:
data = test_1.dropna(subset=['finalPrice'])

In [466]:
data = data.dropna(subset=['description'])

In [469]:
data = data.dropna(subset=['barcode'])

In [473]:
data.columns

Index(['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'originalMetaBriteBarcode',
       'originalMetaBriteDescription', 'pointsNotAwardedReason',
       'pointsPayerId', 'rewardsGroup', 'rewardsProductPartnerId', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber', 'needsFetchReviewReason',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'userFlaggedDescription', 'deleted',
       'priceAfterCoupon', 'metabriteCampaignId'],
      dtype='object')

In [486]:
check = data[(data['needsFetchReview'] != True) &  (data['needsFetchReview'] != False)]

In [491]:
check['needsFetchReview'].replace(np.nan, False)

1       False
2       False
5       False
6       False
9       False
        ...  
6937    False
6938    False
6939    False
6940    False
0       False
Name: needsFetchReview, Length: 2817, dtype: bool

In [ ]:
receiptitems_cols = ['barcode','description','finalPrice','itemPrice','quantityPurchased', 'partnerItemId','needsFetchReview']

In [406]:
receipt_df = pd.read_csv('csv_files/receipts.csv', index_col = 0)

receipts_df = convertDates(receipt_df)
receipts_df['_id'] = cleanIdCpg(receipts_df, column = '_id')
receipts_df = receipts_df.rename(columns = {'_id':'receipt_id'}, inplace = False)
col_list = list(receipts_df.columns)
cols = col_list[3:10]

for col in cols:
    receipts_df[col] = pd.to_datetime(receipts_df[col])
amounts = range(len(receipts_df['rewardsReceiptItemList'].to_dict()))


fullDataSet = []

for j in amounts:
    try:
        text = receipts_df['rewardsReceiptItemList'].to_dict()[j]
        # creating the regex pattern & use re.sub()
        # [\([{})\]] is a RE pattern for selecting
        # '{', '}', '[', ']', '(', ')' brakcets.
        patn = re.sub(r"[\({}[)\]]", "", text)
        split_dictionary = re.findall("'([^}]*)'", patn)[0].split(',')
    except:
        None

        fullKeysValues = {}
        
        for i in range(len(split_dictionary)):
            try:
                keyvalueSplit = split_dictionary[i].split(':')
                
                dataKeys = keyvalueSplit[0].strip(" ' ")
                dataValues = keyvalueSplit[1].strip(" ' ")
                if dataValues == 'None':
                    fullKeysValues[f'{dataKeys}_none']  = dataValues
                else:
                    fullKeysValues[dataKeys]  = dataValues
                    
            except IndexError:
                None
            else:
                dataValues = "None"
                dataKeys = keyvalueSplit[0].strip(" ' ")
                if dataValues == 'None':
                    fullKeysValues[f'{dataKeys}_none']  = dataValues
                else:
                    fullKeysValues[dataKeys]  = dataValues

        fullDataSet.append(fullKeysValues)
fupa = pd.DataFrame(fullDataSet)
fupa

,barcode,barcode_none,description,description_none,finalPrice,finalPrice_none,itemPrice,itemPrice_none,partnerItemId,partnerItemId_none,...,metabriteCampaignId,metabriteCampaignId_none,pointsNotAwardedReason,pointsNotAwardedReason_none,itemNumber,itemNumber_none,deleted,deleted_none,originalMetaBriteQuantityPurchased,originalMetaBriteQuantityPurchased_none
0,021000068364,None,JUST CRACK AN EGG Scramble Kit,None,9.99,None,9.99,None,5,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,013000993906,None,HEINZ Cleaning Vinegar,None,1,None,1,None,1,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,034100573065,None,MILLER LITE 24 PACK 12OZ CAN,None,1.00,None,1.00,None,4,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10 CLASSIC COMBO J,None,3.09,None,3.09,None,1011,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,flipbelt level terrain waist pouch,None,28.57,None,28.57,None,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,B07BRRLSVC,None,thindust summer face mask - sun protection nec...,None,11.99,None,11.99,None,1,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
436,B07BRRLSVC,None,thindust summer face mask - sun protection nec...,None,11.99,None,11.99,None,1,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
437,B07BRRLSVC,None,thindust summer face mask - sun protection nec...,None,11.99,None,11.99,None,1,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
438,B07BRRLSVC,None,thindust summer face mask - sun protection nec...,None,11.99,None,11.99,None,1,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [410]:
len(fupa['description'].unique())

39

# Brand Data Cleaning

In [444]:
def BrandIdCpg(id_data, column=None):
#     print(id_data[f'{column}'])
    # create the dictionary into a list to only get the values
    id_list = list(id_data[f'{column}'].to_dict().values())
    cleanedList = []
    for i in range(len(id_list)):
        parsed_name = id_list[i]
        # remove commas colons and split at quotations
        parsed_names = re.findall("'([^']*)'", id_list[i])
        # these were reoccuring strings that i saw
        remove_list = ['$ref', 'Cogs','$id', '$oid']
        # remove the extra strings and keep the actual id #
        res = [i for i in parsed_names if i not in remove_list]
        cleanedList.append(res[0])
        
    return cleanedList

In [1017]:
brand_df = pd.read_csv('csv_files/brands.csv', index_col = 0)

In [1084]:
brand_df = pd.read_csv('csv_files/brands.csv', index_col = 0)
brands_df = brand_df.copy()
brands_df['_id'] = BrandIdCpg(brands_df, column = '_id')
brands_df['cpg'] = BrandIdCpg(brands_df, column = 'cpg')
brands_df = brands_df.rename(columns = {'_id':'brand_id'}, inplace = False)
len(brands_df)

1167

In [1106]:
brands_df = brands_df[(brands_df['name'].str.contains("test") == False)]

In [1113]:
brands_df['topBrand'].isna().value_counts()

True     407
False    331
Name: topBrand, dtype: int64

In [1149]:
brands_df

,brand_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,0.0,STARBUCKS
7,5cdad0f5166eb33eb7ce0faa,511111104810,Condiments & Sauces,NaN,559c2234e4b06aca36af13c6,J.L. Kraft,NaN,J.L. KRAFT
8,5ab15636e4b0be0a89bb0b07,511111504412,Canned Goods & Soups,NaN,5a734034e4b0d58f376be874,Campbell's Home Style,0.0,CAMPBELLS HOME STYLE
11,57c08106e4b0718ff5fcb02c,511111102540,NaN,NaN,Cpgs,MorningStar,NaN,NaN
12,588ba07be4b02187f85cdadd,511111201076,Baking,NaN,559c2234e4b06aca36af13c6,Calumet,0.0,CALUMET
...,...,...,...,...,...,...,...,...
1159,585a96cbe4b03e62d1ce0e88,511111501619,Beverages,NaN,5332f5fbe4b03c9a25efd0ba,Pepsi Max,0.0,NaN
1160,5887a216e4b02187f85cdad5,511111401155,Deli,NaN,559c2234e4b06aca36af13c6,Claussen,0.0,CLAUSSEN
1161,5332f709e4b03c9a25efd0f2,511111403845,Beer Wine Spirits,NaN,5332f709e4b03c9a25efd0f1,Blue Moon,0.0,BLUE MOON
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,53e10d6368abd3c7065097cc,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL


## Data Analysis

In [1140]:
users_df['_id'].value_counts()

5fc961c3b8cfca11a077dd33    20
54943462e4b07e684157a532    20
5ff5d15aeb7c7d12096d91a2    18
59c124bae4b0299e55b0f330    18
5fa41775898c7a11a6bcef3e    18
                            ..
600258dafb296c4ef805d9b7     1
5ff1e1e8cfcf6c399c274ad9     1
601465a584231211ce796da5     1
60268c69efa6011bb151075f     1
5ffc9001b3348b11c93388b6     1
Name: _id, Length: 212, dtype: int64

In [938]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   _id           495 non-null    object        
 1   active        495 non-null    bool          
 2   createdDate   495 non-null    datetime64[ns]
 3   lastLogin     433 non-null    datetime64[ns]
 4   role          495 non-null    object        
 5   signUpSource  447 non-null    object        
 6   state         439 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 27.6+ KB


In [936]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     898 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 82.1+ KB


In [1144]:
receipts_df['userId'].value_counts()

5fc961c3b8cfca11a077dd33    436
59c124bae4b0299e55b0f330     58
54943462e4b07e684157a532     50
5fa41775898c7a11a6bcef3e     21
5ff5d15aeb7c7d12096d91a2     20
                           ... 
6009885c50b33111fd61eee9      1
5ffc8ff9b3348b11c9338896      1
5ff8da7c04929111f6e914de      1
5fa8d542b055fd1dfdf09517      1
600b41d050b3311194385655      1
Name: userId, Length: 258, dtype: int64

In [1079]:
receipts_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   receipt_id               1119 non-null   object        
 1   bonusPointsEarned        544 non-null    float64       
 2   bonusPointsEarnedReason  544 non-null    object        
 3   createDate               1119 non-null   datetime64[ns]
 4   dateScanned              1119 non-null   datetime64[ns]
 5   finishedDate             568 non-null    datetime64[ns]
 6   modifyDate               1119 non-null   datetime64[ns]
 7   pointsAwardedDate        537 non-null    datetime64[ns]
 8   pointsEarned             609 non-null    datetime64[ns]
 9   purchaseDate             671 non-null    datetime64[ns]
 10  purchasedItemCount       635 non-null    float64       
 11  rewardsReceiptItemList   679 non-null    object        
 12  rewardsReceiptStatus     1119 non-

In [ ]:
What are the top 5 brands by receipts scanned for most recent month?
How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
Which brand has the most spend among users who were created within the past 6 months?
Which brand has the most transactions among users who were created within the past 6 months?

In [1134]:
len(brands_df)

738

186

In [971]:
receipts_df['dateScanned'].min()
receipts_df['dateScanned'].max()

Timestamp('2021-03-01 00:00:00')

In [1120]:
recentMonth = receipts_df[receipts_df['dateScanned'] == receipts_df['dateScanned'].max() ] 

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
1089,603cbbb50a720fde100003e3,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,1970-01-01 00:00:00.000000025,2020-08-17,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1090,603cadab0a720fde100003d6,NaN,NaN,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,NaT,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1091,603c8cdf0a7217c72c0003bb,NaN,NaN,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,NaT,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1092,603d30e60a7217c72c00043f,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,1970-01-01 00:00:00.000000025,2020-08-17,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1093,603c65060a720fde10000395,NaN,NaN,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,NaT,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1094,603d5d6c0a7217c72c000463,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,1970-01-01 00:00:00.000000025,2020-08-17,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1095,603d760e0a720fde1000048e,NaN,NaN,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,NaT,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1096,603d59e70a7217c72c00045f,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,1970-01-01 00:00:00.000000025,2020-08-17,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1097,603c5ba30a720fde1000038d,NaN,NaN,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,NaT,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1098,603c51160a7217c72c00038b,NaN,NaN,2021-03-01,2021-03-01,NaT,2021-03-01,NaT,NaT,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
